In [1]:
import os
import pyodbc
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import (
    HistGradientBoostingRegressor,
    HistGradientBoostingClassifier,
)
from sklearn.model_selection import cross_validate, train_test_split

**Setup**

In [3]:
SEED = 1729
sns.set_theme(style='whitegrid')

# Space Challenge Analysis Notebook

## Connect to database
The present working directory should contain a file `.env` with the line `SPACECHALLENGE_DB_CONN_STR='{your db connecion string}'`.

In [4]:
load_dotenv()
db_conn_str = os.environ['SPACECHALLENGE_DB_CONN_STR']

In [5]:
# confirm everythings working
# with pyodbc.connect(db_conn_str) as conn:
#     with conn.cursor() as cursor:
#         df = pd.read_sql('select top 10 * from bookings', conn)
# df

## Load dataset

In [7]:
dataset_query = """
select
    ah.AssignmentID, b.BookingID, sta.AgentID
    -- dependent var
    ,b.PackageRevenue
    ,b.BookingStatus
    -- vars we know about customer
    ,ah.CustomerName, ah.CommunicationMethod, ah.LeadSource, b.Destination, b.LaunchLocation
    -- vars we know about agent
    ,sta.JobTitle, sta.DepartmentName, sta.ManagerName, sta.YearsOfService, sta.AverageCustomerServiceRating
    -- datetime vars
    ,ah.AssignedDateTime, b.BookingCompleteDate, b.CancelledDate
    -- other rev values
    ,b.DestinationRevenue, b.TotalRevenue
from assignment_history ah
left join bookings b
    on ah.AssignmentID = b.AssignmentID
left join space_travel_agents sta
    on sta.AgentID = ah.AgentID
order by ah.AssignmentID
"""
with pyodbc.connect(db_conn_str) as conn:
    with conn.cursor() as cursor:
        dataset_df = pd.read_sql(dataset_query, conn)

/var/folders/9h/8rdmtj8j6g17zl5bvhyvtd9h0000gn/T/ipykernel_37168/1657903215.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset_df = pd.read_sql(dataset_query, conn)


In [8]:
dataset_df.head(10)

,AssignmentID,BookingID,AgentID,PackageRevenue,BookingStatus,CustomerName,CommunicationMethod,LeadSource,Destination,LaunchLocation,JobTitle,DepartmentName,ManagerName,YearsOfService,AverageCustomerServiceRating,AssignedDateTime,BookingCompleteDate,CancelledDate,DestinationRevenue,TotalRevenue
0,1,1.0,7,25000.0,Confirmed,Evelyn Brooks,Phone Call,Organic,Mars,Dallas-Fort Worth Launch Complex,Space Travel Agent,Interplanetary Sales,Lyra Chen,4,4.5,2081-02-01 09:00:00,2081-02-01 10:00:00,NaT,150000.0,175000.0
1,2,2.0,14,20000.0,Cancelled,Lucas Chen,Text,Bought,Europa,New York Orbital Gateway,Lead Space Travel Agent,Luxury Voyages,Zane Holloway,16,4.2,2081-02-01 10:30:00,NaT,2081-02-01 11:00:00,120000.0,140000.0
2,3,NaN,3,NaN,None,Amara Patel,Text,Organic,None,None,Senior Space Travel Agent,Premium Bookings,Zane Holloway,15,4.0,2081-02-01 11:15:00,NaT,NaT,NaN,NaN
3,4,3.0,25,30000.0,Confirmed,Jasper Kim,Phone Call,Bought,Titan,Dubai Interplanetary Hub,Space Travel Agent,Interplanetary Sales,Lyra Chen,5,4.4,2081-02-01 13:00:00,2081-02-01 13:30:00,NaT,140000.0,170000.0
4,5,4.0,7,25000.0,Cancelled,Zara Singh,Text,Organic,Mars,Dallas-Fort Worth Launch Complex,Space Travel Agent,Interplanetary Sales,Lyra Chen,4,4.5,2081-02-01 14:20:00,NaT,2081-02-01 15:10:00,150000.0,175000.0
5,6,5.0,17,25000.0,Confirmed,Milo Davis,Phone Call,Bought,Venus,Tokyo Spaceport Terminal,Space Travel Agent,Luxury Voyages,Lyra Chen,3,3.3,2081-02-02 09:45:00,2081-02-02 10:00:00,NaT,130000.0,155000.0
6,7,NaN,9,NaN,None,Aurora Ramirez,Text,Organic,None,None,Space Travel Agent,Premium Bookings,Zane Holloway,6,4.4,2081-02-02 10:10:00,NaT,NaT,NaN,NaN
7,8,6.0,14,30000.0,Confirmed,Theo White,Phone Call,Organic,Mars,New York Orbital Gateway,Lead Space Travel Agent,Luxury Voyages,Zane Holloway,16,4.2,2081-02-02 11:55:00,2081-02-02 12:00:00,NaT,150000.0,180000.0
8,9,7.0,28,20000.0,Pending,Isla Nguyen,Text,Bought,Europa,London Ascension Platform,Space Travel Agent,Interplanetary Sales,Lyra Chen,6,4.9,2081-02-02 13:40:00,NaT,NaT,125000.0,145000.0
9,10,8.0,1,15000.0,Confirmed,Rowan Scott,Phone Call,Organic,Ganymede,Sydney Stellar Port,Senior Space Travel Agent,Interplanetary Sales,Lyra Chen,12,4.0,2081-02-02 15:00:00,2081-02-02 15:30:00,NaT,110000.0,125000.0


In [8]:
dataset_df.columns

Index(['AssignmentID', 'BookingID', 'AgentID', 'PackageRevenue',
       'BookingStatus', 'CustomerName', 'CommunicationMethod', 'LeadSource',
       'Destination', 'LaunchLocation', 'JobTitle', 'DepartmentName',
       'ManagerName', 'YearsOfService', 'AverageCustomerServiceRating',
       'AssignedDateTime', 'BookingCompleteDate', 'CancelledDate',
       'DestinationRevenue', 'TotalRevenue'],
      dtype='object')

## Single Gradient Boosting model
Basic prediction at agent level of `ActualizedRevenue` := `PackageRevenue` if `'Confirmed'` else `0`.
* Ignore Assignments that don't have a related Booking
* Drop `Pending` Bookings.

**Details**
|  |  |
|--|--|
| Model | [`HistGradientBoostingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#histgradientboostingregressor) |
| Dependent Variable | Agent's `ActualizedRevenue` := `PackageRevenue` if `'Confirmed'` else `0` |
| Independent Variables | `['CommunicationMethod', 'LeadSource', 'Destination', 'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName', 'YearsOfService', 'AverageCustomerServiceRating']` |

In [9]:
# drop where no booking data or booking pending
df = dataset_df[~(dataset_df['BookingID'].isna() | (dataset_df['BookingStatus'] == 'Pending'))].reset_index()

categorical_vars = [
    'CommunicationMethod', 'LeadSource', 'Destination', 
    'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName'
]
numerical_vars = ['YearsOfService', 'AverageCustomerServiceRating']

X = df[categorical_vars + numerical_vars]

y = df['PackageRevenue'].where(df['BookingStatus']=='Confirmed', 0.0)

gbr_model = HistGradientBoostingRegressor(
    loss='poisson',  # because y non-negative
    categorical_features=categorical_vars,
    random_state=SEED,
)

print(f'Num obs: {X.shape[0]}')
print(f'{X.shape[1]} features. {len(categorical_vars)} categorical, {len(numerical_vars)} numerical.')

Num obs: 392
9 features. 7 categorical, 2 numerical.


In [10]:
%%time
result = cross_validate(gbr_model, X, y)

CPU times: user 1.98 s, sys: 133 ms, total: 2.11 s
Wall time: 3.98 s


In [11]:
result
# no good

{'fit_time': array([2.88243794, 0.22455668, 0.3877871 , 0.16787815, 0.21736932]),
 'score_time': array([0.01702285, 0.0442121 , 0.00869918, 0.00753284, 0.01153159]),
 'test_score': array([-0.20425296, -0.14666003, -0.50537806, -0.1697015 , -0.10366318])}

## Predicting only `'Confirmed'` `PackageRevenue`
Basic prediction at agent level of `PackageRevenue` if `'Confirmed'`.
* Ignore Assignments that don't have a related Booking
* Drop `'Pending'` and `'Cancelled'` Bookings.

**Details**
|  |  |
|--|--|
| Model | [`HistGradientBoostingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#histgradientboostingregressor) |
| Dependent Variable | Agent's `PackageRevenue` if `'Confirmed'` |
| Independent Variables | `['CommunicationMethod', 'LeadSource', 'Destination', 'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName', 'YearsOfService', 'AverageCustomerServiceRating']` |

In [12]:
# drop where no booking data or booking pending
df = dataset_df[dataset_df['BookingStatus'] == 'Confirmed'].reset_index()

categorical_vars = [
    'CommunicationMethod', 'LeadSource', 'Destination', 
    'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName'
]
numerical_vars = ['YearsOfService', 'AverageCustomerServiceRating']

X = df[categorical_vars + numerical_vars]

y = df['PackageRevenue']

gbr_model = HistGradientBoostingRegressor(
    loss='poisson',  # because y non-negative
    categorical_features=categorical_vars,
    random_state=SEED,
)

print(f'Num obs: {X.shape[0]}')
print(f'{X.shape[1]} features. {len(categorical_vars)} categorical, {len(numerical_vars)} numerical.')

Num obs: 297
9 features. 7 categorical, 2 numerical.


In [13]:
%%time
result = cross_validate(gbr_model, X, y)

CPU times: user 1.03 s, sys: 29 ms, total: 1.06 s
Wall time: 617 ms


In [14]:
result
# no good

{'fit_time': array([0.14040399, 0.10178804, 0.09709883, 0.13653135, 0.09704399]),
 'score_time': array([0.00750279, 0.00721502, 0.01005101, 0.00697398, 0.00701904]),
 'test_score': array([-0.13952736, -0.56033317,  0.2147542 ,  0.08852657,  0.19851088])}

## Classifying `PackageRevenue` when `'Confirmed'` 
`PackageRevenue` is in increments of 5k from 0 to 30k. 
Let's try to classify if it's 30k, 25k, 20k, >=15k. 
This will put >=29 obs / class with least in the >=15k class. 

* Ignore Assignments that don't have a related Booking
* Drop `'Pending'` and `'Cancelled'` Bookings.

**Details**
|  |  |
|--|--|
| Model | [`HistGradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) |
| Dependent Variable | Agent's `PackageRevenue` if `'Confirmed'` bucketed as explained above |
| Independent Variables | `['CommunicationMethod', 'LeadSource', 'Destination', 'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName', 'YearsOfService', 'AverageCustomerServiceRating']` |

In [15]:
# drop where no booking data or booking pending
df = dataset_df[dataset_df['BookingStatus'] == 'Confirmed'].reset_index()

In [ ]:
categorical_vars = [
    'CommunicationMethod', 'LeadSource', 'Destination', 
    'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName'
]
numerical_vars = ['YearsOfService', 'AverageCustomerServiceRating']

X = df[categorical_vars + numerical_vars]

y = df['PackageRevenue'].where(df['PackageRevenue'] > 15000, 15000)

gbr_model = HistGradientBoostingClassifier(
    learning_rate=0.3,
    categorical_features=categorical_vars,
    min_samples_leaf=10,  # since such small sample size
    random_state=SEED,
)

print(f'Num obs: {X.shape[0]}')
print(f'{X.shape[1]} features. {len(categorical_vars)} categorical, {len(numerical_vars)} numerical.')

In [22]:
%%time
result = cross_validate(gbr_model, X, y)

CPU times: user 5.28 s, sys: 99.6 ms, total: 5.38 s
Wall time: 2.92 s


In [23]:
result
# better only in relation to the ones above

{'fit_time': array([0.62742805, 0.52521706, 0.5075562 , 0.61850286, 0.56614828]),
 'score_time': array([0.01590395, 0.01155996, 0.01201415, 0.01203775, 0.01139784]),
 'test_score': array([0.26666667, 0.38333333, 0.44067797, 0.49152542, 0.40677966])}

## Classifying `PackageRevenue` when `'Confirmed'` w/o kitchen sink
`PackageRevenue` is in increments of 5k from 0 to 30k. 
Let's try to classify if it's 30k, 25k, 20k, >=15k. 
This will put >=29 obs / class with least in the >=15k class. 

* Including only select indep vars
* Ignore Assignments that don't have a related Booking
* Drop `'Pending'` and `'Cancelled'` Bookings.

**Details**
|  |  |
|--|--|
| Model | [`HistGradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) |
| Dependent Variable | Agent's `PackageRevenue` if `'Confirmed'` bucketed as explained above |
| Independent Variables | `['Destination', 'AgentID', 'DepartmentName', 'YearsOfService', 'AverageCustomerServiceRating']` |

In [24]:
# drop where no booking data or booking pending
df = dataset_df[dataset_df['BookingStatus'] == 'Confirmed'].reset_index()

In [29]:
categorical_vars = ['Destination', 'AgentID', 'DepartmentName']
numerical_vars = ['YearsOfService', 'AverageCustomerServiceRating']

X = df[categorical_vars + numerical_vars]

y = df['PackageRevenue'].where(df['PackageRevenue'] > 15000, 15000)

gbr_model = HistGradientBoostingClassifier(
    learning_rate=0.3,
    categorical_features=categorical_vars,
    min_samples_leaf=10,  # since such small sample size
    random_state=SEED,
)

print(f'Num obs: {X.shape[0]}')
print(f'{X.shape[1]} features. {len(categorical_vars)} categorical, {len(numerical_vars)} numerical.')

Num obs: 297
5 features. 3 categorical, 2 numerical.


In [30]:
%%time
result = cross_validate(gbr_model, X, y)

CPU times: user 4.77 s, sys: 54.1 ms, total: 4.83 s
Wall time: 2.56 s


In [32]:
result

{'fit_time': array([0.56958485, 0.50992012, 0.48254108, 0.46549797, 0.46697307]),
 'score_time': array([0.01083612, 0.01534891, 0.01117015, 0.01051474, 0.01043701]),
 'test_score': array([0.15      , 0.21666667, 0.44067797, 0.42372881, 0.25423729])}

## Classifying `PackageRevenue` when `'Confirmed'` 
`PackageRevenue` is in increments of 5k from 0 to 30k. 
Let's try to classify if it's 30k, 25k, 20k, >=15k. 
This will put >=29 obs / class with least in the >=15k class. 

* Ignore Assignments that don't have a related Booking
* Drop `'Pending'` and `'Cancelled'` Bookings.

**Details**
|  |  |
|--|--|
| Model | [`HistGradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) |
| Dependent Variable | Agent's `PackageRevenue` if `'Confirmed'` bucketed as explained above |
| Independent Variables | `['CommunicationMethod', 'LeadSource', 'Destination', 'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName', 'YearsOfService', 'AverageCustomerServiceRating']` |

In [15]:
# drop where no booking data or booking pending
df = dataset_df[dataset_df['BookingStatus'] == 'Confirmed'].reset_index()

In [ ]:
categorical_vars = [
    'CommunicationMethod', 'LeadSource', 'Destination', 
    'LaunchLocation', 'AgentID', 'JobTitle', 'DepartmentName'
]
numerical_vars = ['YearsOfService', 'AverageCustomerServiceRating']

X = df[categorical_vars + numerical_vars]

y = df['PackageRevenue'].where(df['PackageRevenue'] > 15000, 15000)

gbr_model = HistGradientBoostingClassifier(
    learning_rate=0.3,
    categorical_features=categorical_vars,
    min_samples_leaf=10,  # since such small sample size
    random_state=SEED,
)

print(f'Num obs: {X.shape[0]}')
print(f'{X.shape[1]} features. {len(categorical_vars)} categorical, {len(numerical_vars)} numerical.')

In [22]:
%%time
result = cross_validate(gbr_model, X, y)

CPU times: user 5.28 s, sys: 99.6 ms, total: 5.38 s
Wall time: 2.92 s


In [23]:
result
# better only in relation to the ones above

{'fit_time': array([0.62742805, 0.52521706, 0.5075562 , 0.61850286, 0.56614828]),
 'score_time': array([0.01590395, 0.01155996, 0.01201415, 0.01203775, 0.01139784]),
 'test_score': array([0.26666667, 0.38333333, 0.44067797, 0.49152542, 0.40677966])}